AAICではNQSIIというバッチジョブスケジューラが動作しています。<br>
長時間実行する計算ジョブはNQSIIへジョブを投入する必要があります。<br>

ここでは、ChainerMNでMNIST(手書き数字認識)のサンプルのジョブをNQSIIへ投入して動かしてみます。<br>
まず、手元にChainerMNのソースコードがない場合は、ChainerMNのGitHubからソースコードをダウンロードします

In [ ]:
curl -L https://github.com/chainer/chainermn/archive/v1.0.0b2.tar.gz | tar zx

chainermn-1.0.0b2というディレクトリが作成されており、MNISTのサンプルがchainermn-1.0.0b2/examples/mnist以下にあることを確認します。

In [ ]:
ls chainermn-1.0.0b2/examples/mnist

バッチジョブスケジューラへジョブを投入するためには、まず、スクリプトファイルを記述する必要があります。<br>
詳細は以下を参照してください。<br>
https://github.com/aistairc/aaic/wiki/manual/NQSII_UsersGuide-Operation.pdf


share/submit_chainermn.nqにtrain_mnist.pyをMPIで実行するためのサンプルのスクリプトファイルがあります。

In [ ]:
cat share/submit_chainermn.nqs

NQSIIに関連する記述は以下の通りです。
* \#PBS --group=[AAICグループ名]<br>投入するジョブが実行する際のAAICグループを指定します。ここではg-nairobiというAAICグループを指定していますが、適切な自分のAAICグループを指定してください。<br>
* \#PBS -q [キュー名]<br>ジョブを投入するキューを指定します。ここではgqを指定しています。現在のAAICでのキュー構成は以下のサイトを参照してください。<br>
https://github.com/aistairc/aaic/wiki/AAIC仕様・現在利用可能サービス <br>
* \#PBS -b [計算ノードの台数]<br>計算ノードの数を指定します。ここでは、2台の計算ノードを指定しています。
* \#PBS -l cpunum_job=40, \#PBS -l gpunum_job=8<br>
CPUのコア数とGPUの台数を指定します。AAICのGPUを搭載した計算ノードでは、下図のように、1台あたり8つのGPUを搭載しており、CPUは40コア動作しているようにみえます。MPIを用いて安定で高速に計算する際は計算ノードを専有して使用する必要があるため、このような宣言が必要です。<br>
* \#PBS -l elapstim_req=[実行時間]<br>
ジョブが動作する実行時間を指定します。ここでは5分と宣言しています。AAICでは短めの実行時間を宣言したほうがジョブがスケジュールされやすい設定になっています。(正確には、宣言された実行時間と実際に実行した際の実行時間の乖離が小さい場合が望ましい。)<br>
* \#PBS -T openmpi, \#PBS -v NQSII_OMPI_MODVER=1.10.4/gcc4.8.5_cuda8.0<br>
OpenMPIを用いてMPIを実行する際に必要になるオプションです。
* \#PBS -N [ジョブ名]<br>
ジョブに名前をつけることができます。ジョブ実行中の標準出力や標準エラー出力が出力されるファイルの名前のプリフィックスなどに用いられます。<br>

cd \${PBS_O_WORKDIR}でqsubコマンドを実行したカレントディレクトリに移動して、その後、MPIを起動しています。<br>
ここで指定しているmpirunのオプションは以下のとおりです。<br>
* \-x PATH<br>
現在のPATH環境変数の内容をMPIで実行するリモートの計算ノードでも同様に指定します。
* \-n [数値]<br>
並列に実行するプロセス数を指定することができます。ここでは、1ノードあたり8GPU搭載していて、それを2ノードで実行するので16と指定しています。<br>
* \${NQSII_MPIOPS}<br>
NQSIIが指定するオプションを認識させるためのおまじないです。実際にはMPIのプロセスを実行する計算ノードを記載したファイルなどが設定されます。<br>
* \--npersocket 4<br>
1ソケット(CPU)あたりに動作させる並列数を指定します。AAICでは下記のように1CPUあたりに4GPUが接続されているのでこのような設定を入れたほうが望ましいです。<br>

![GPUNode](share/gpx001.pdf)

qsubコマンドを用いてNQSIIへジョブを投入します。

In [ ]:
qsub share/submit_chainermn.nqs

qstatとすると実行中のジョブの状態を確認することができます。

In [ ]:
man qstat 

In [ ]:
qstat -f 38883.bsv001

RequestIDが投入したジョブに与えられたIDです。<br>
投入したジョブの状態(qstatコマンド)や投入したジョブの出力(qcatコマンド)、投入したジョブの削除(qdelコマンド)などの操作の際に必要になります。<br>
簡単のため、ここでは以下のようにRequestIDを取得することにします。<br>

In [ ]:
RequestID=`qstat | awk 'NR==3' | awk -F' ' '{print $1}'`
echo $RequestID

STTに投入したジョブの状態が示されます。QUEの場合は実行待ち状態、RUNの場合は実行中になります。<br>

qcatコマンドにより投入したジョブのスクリプトの内容や出力結果を確認することができます。

In [ ]:
man qcat

ジョブスクリプトの内容を出力します。 (オプションなし)

In [ ]:
qcat $RequestID

標準出力の内容を出力します。（-oオプション)

In [ ]:
qcat -o $RequestID

標準エラー出力の内容を出力します。(-eオプション)

In [ ]:
qcat -e $RequestID

qdelコマンドにより投入したジョブを削除することができます。<br>
今回は実際にはジョブを削除しません。

In [ ]:
man qdel

実行中の標準出力、標準エラー出力の結果はそれぞれ[ジョブ名].o[RequestIDの数値の部分], [ジョブ名].e[RequestIDの数値の部分]というファイルに格納されます。<br>
例えば、今回はジョブ名をchainermn-testと設定しているので、chainermn-test.o[RequestIDの数値の部分], chainermn-test.e[RequestIDの数値の部分]などに出力されます。

今、RequestIDの数値の部分をJobIDとし、以下のように取得します。

In [ ]:
export JobID=${RequestID:0:-7}
echo $JobID

標準出力の内容を出力します。

In [ ]:
cat chainermn-test.o$JobID

標準エラー出力の内容を出力します。

In [ ]:
cat chainermn-test.e$JobID

resultというディレクトリにtrain_mnist.pyの実行結果が出力されます。

In [ ]:
ls result

cg.dotはDOT言語で記述されたネットワーク構造のファイル、logはJSONで記述された実行時間、エポック数、反復回数、精度などを記述したファイルになります。